In [1]:
# Code to load a trained model and test on file with vibrations

In [1]:
import sys

sys.path.append('..')
sys.path.append('../..')

In [2]:

import json
import pickle as pkl

from directory_paths import MODELS_DIR, PARENT_DIR, SAVE_DIR
from inference.test_model_utils import Tester, parse_config_value
from model.model_definitions import SpatiotemporalNetwork as net
from train.new_spindle_dataset import SpindleDataset
from train.train_model_utils import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from utils.muscle_names import MUSCLE_NAMES

In [3]:
# task = "letter_reconstruction_joints_vel"
task = "letter_reconstruction_joints"
model_path = os.path.join(
    SAVE_DIR,
    "trained_models/experiment_causal_flag-pcr_optimizedLinearFR_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8-8-32-64_7171_0",
)
# model_path = os.path.join(
#     SAVE_DIR,
#     f"trained_models/experiment_causal_flag-pcr_optimized_linearFR_extended_5_5_30k_noLayerNorm_elbowAngles_stride1_240Hz_{task}/spatiotemporal_4_8-8-32-64_7171_0_0",
# )
INPUT_DATA = "ELBOW"
SAMPLE_RATE = "240Hz"
# test_exp_dir = "vib_multiple_freqs"
test_exp_dir = "vib_vary_multipleFs_addNoVibOutput"
# test_exp_dir = "vib_vary_multipleFs_vib_ii_only"
# test_exp_dir = "vib_vary_multipleFs_vib_ia"
# test_exp_dir = "vib_vary_multipleFs"

# /media/data4/adriana/ProprioPerception/trained_models/experiment_causal_flag-pcr_optimizedLinearFR_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8-8-32-64_7171_0/test/ELBOW/240Hz/vib_multiple_freqs/vib_0-190Hz_5chan_2biceps
PATH_TO_SAVE = model_path + "/" + "test" + "/" + INPUT_DATA + "/" + SAMPLE_RATE + "/" + test_exp_dir
# PATH_TO_SAVE = PATH_TO_SAVE + "/vib_0-190Hz_5chan_1Trilat"
# PATH_TO_SAVE = PATH_TO_SAVE + "/vib_0_190Hz_5channels_biceps_FRmin"
# PATH_TO_SAVE = PATH_O_SAVE + "/vib_0-190Hz_5chan_3triceps"
# PATH_TO_SAVE = PATH_TO_SAVE + "/vib_0-190Hz_5chan_1bicepsLong"
# PATH_TO_SAVE = PATH_TO_SAVE + "/vib_0-190Hz_5chan_1biceps"

# path_to_df = PATH_TO_SAVE + "/vib_0-190Hz_5chan_2biceps"
# path_to_results = os.path.join(path_to_df, "vib_results.h5")
# df_all = pd.read_hdf(path_to_results)


# Initialize an empty list to store DataFrames
dataframes = []

# Iterate over all subdirectories in PATH_TO_SAVE
for subdir in os.listdir(PATH_TO_SAVE):
    print(subdir)
    subdir_path = os.path.join(PATH_TO_SAVE, subdir)
    if not os.path.isdir(subdir_path):  # Ensure it's a directory
        continue
    for subdir2 in os.listdir(subdir_path):
        print(subdir2)
        subdir_path = os.path.join(subdir_path, subdir2)
        if os.path.isdir(subdir_path):  # Ensure it's a directory
            h5_file_path = os.path.join(subdir_path, "vib_results.h5")
            if os.path.exists(h5_file_path):  # Check if the HDF5 file exists
                print(f"Loading {h5_file_path}")
                df = pd.read_hdf(h5_file_path)  # Load the HDF5 file
                dataframes.append(df)  # Append the DataFrame to the list

# Concatenate all loaded DataFrames into a single DataFrame
df_all = pd.concat(dataframes, ignore_index=True)

print(f"Loaded DataFrame shape: {df_all.shape}")

plot_elbow_angles_trial_65_150_TRIlat.svg
plot_elbow_angles_trial_65_150_BIClong.svg
plot_elbow_angles_trial_82_150_TRIlat.svg
plot_elbow_angles_trial_12_150_TRIlat.svg
plot_elbow_angles_trial_62_150_TRIlat.svg
plot_elbow_angles_trial_97_150_BIClong.svg
plot_elbow_angles_trial_61_150_TRIlat.svg
predElbow_trial_61_muscle_4.svg
TRIlat_TRIlong_TRImed
vib_0-190Hz_5chan_3triceps
Loading /media/data/merkourios/ProprioPerception/trained_models/experiment_causal_flag-pcr_optimizedLinearFR_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8-8-32-64_7171_0/test/ELBOW/240Hz/vib_vary_multipleFs_addNoVibOutput/TRIlat_TRIlong_TRImed/vib_0-190Hz_5chan_3triceps/vib_results.h5
outputs_6_perFeq_triceps.svg
inputs_vs_vibfreq_TRIlat.svg
BIClong
vib_0-190Hz_5chan_1BIClong
Loading /media/data/merkourios/ProprioPerception/trained_models/experiment_causal_flag-pcr_optimizedLinearFR_30k_noLayerNorm_elbowAngles_stride1_240Hz_letter_reconstruction_joints/spatiotemporal_4_8-8

In [4]:
ELBOW_ANGLE_INDEX = 6
TIME = 1152
sample_rate = 240
muscle_groups = [["BIClong", "BICshort"], ["BIClong"], ["TRIlat", "TRIlong", "TRImed"], ["TRIlat"]]

In [9]:
def test_angle_offset(df, muscle_group, elbow_angle_range, vib_freq, direction="greater"):
    from scipy import stats
    data = df[
        (df["elbow_angle"] >= elbow_angle_range[0]) 
        & (df["elbow_angle"] <= elbow_angle_range[1])
        & (df["vib_freq"] == vib_freq)
        & df["vib_muscles"].apply(lambda x: x == muscle_group)
    ]["vib_angle_diff_elbow"]

    print(elbow_angle_range)
    print(data.describe())

    # Test hypothesis that the mean of the data is greater than 0 with a non-parametric test
    test_result = stats.wilcoxon(data, alternative=direction)

    return test_result.statistic, test_result.pvalue

test_results = pd.DataFrame(columns=["muscle", "freq", "angle_range", "negative_p", "positive_p"])
    
for muscle in [["BIClong"], ["TRIlat"]]:
    for freq in [20, 40, 60, 80, 100, 110, 130, 150, 170, 190]:
        for angle_range in [(30,60), (60,90), (90,120), (120,150), (0, 180)]:
            _, p_less = test_angle_offset(df_all, muscle, angle_range, freq, "less")
            _, p_greater = test_angle_offset(df_all, muscle, angle_range, freq, "greater")
            # print(f"{muscle[0]}, {freq}Hz, {angle_range}: {p_less}, {p_greater}")
            test_results = pd.concat(
                [test_results, 
                pd.DataFrame([[muscle[0], freq, angle_range, p_less, p_greater]], columns=test_results.columns)]
            )

test_results.to_csv("figure_3_test_vib_angle_offset.csv", index=False)

(30, 60)
count    17.000000
mean    -15.386462
std       1.681770
min     -17.909325
25%     -16.489349
50%     -14.963821
75%     -14.570663
max     -11.725605
Name: vib_angle_diff_elbow, dtype: float64
(30, 60)
count    17.000000
mean    -15.386462
std       1.681770
min     -17.909325
25%     -16.489349
50%     -14.963821
75%     -14.570663
max     -11.725605
Name: vib_angle_diff_elbow, dtype: float64
(60, 90)
count    32.000000
mean     -9.740307
std       2.894992
min     -14.345894
25%     -11.311786
50%     -10.505190
75%      -8.072075
max      -1.877953
Name: vib_angle_diff_elbow, dtype: float64
(60, 90)
count    32.000000
mean     -9.740307
std       2.894992
min     -14.345894
25%     -11.311786
50%     -10.505190
75%      -8.072075
max      -1.877953
Name: vib_angle_diff_elbow, dtype: float64
(90, 120)
count    26.000000
mean     -4.351326
std       2.628223
min      -9.988167
25%      -6.220831
50%      -4.565292
75%      -2.290600
max       0.752304
Name: vib_angle_diff_e

In [6]:
def test_elbow_ranges(df, muscle_group, vib_freq, range1, range2):
    from scipy import stats
    data1 = df[
        (df["elbow_angle"] >= range1[0]) 
        & (df["elbow_angle"] <= range1[1])
        & (df["vib_freq"] == vib_freq)
        & df["vib_muscles"].apply(lambda x: x == muscle_group)
    ]["vib_angle_diff_elbow"]


    data2 = df[
        (df["elbow_angle"] >= range2[0]) 
        & (df["elbow_angle"] <= range2[1])
        & (df["vib_freq"] == vib_freq)
        & df["vib_muscles"].apply(lambda x: x == muscle_group)
    ]["vib_angle_diff_elbow"]

    # Test hypothesis that data1 and data2 are from the same distribution with a non-parametric test
    test_result = stats.ranksums(data1, data2)

    return test_result.statistic, test_result.pvalue

test_results = pd.DataFrame(columns=["muscle", "freq", "angle_range1", "angle_range2", "p_value"])

for muscle in [["BIClong"], ["TRIlat"]]:
    for freq in [20, 40, 60, 80, 100, 110, 130, 150, 170, 190]:
        for angle_range1 in [(30,60), (60,90), (90,120), (120,150)]:
            for angle_range2 in [(30,60), (60,90), (90,120), (120,150)]:
                if angle_range1 == angle_range2:
                    continue
                _, p = test_elbow_ranges(df_all, muscle, freq, angle_range1, angle_range2)
                test_results = pd.concat(
                    [test_results, 
                    pd.DataFrame([[muscle[0], freq, angle_range1, angle_range2, p]], columns=test_results.columns)]
                )

test_results.to_csv("figure_3_test_vib_angle_ranges.csv", index=False)


In [7]:
def paired_test_vibration_freqs(df, muscle_group, vib_freq_1, vib_freq_2):
    from scipy import stats
    data1 = df[
        (df["vib_freq"] == vib_freq_1)
        & df["vib_muscles"].apply(lambda x: x == muscle_group)
    ][["trial", "vib_angle_diff_elbow"]]

    data2 = df[
        (df["vib_freq"] == vib_freq_2)
        & df["vib_muscles"].apply(lambda x: x == muscle_group)
    ][["trial", "vib_angle_diff_elbow"]]

    merged_data = pd.merge(data1, data2, on="trial", suffixes=("_1", "_2"))
    merged_data["vib_diff"] = merged_data["vib_angle_diff_elbow_2"] - merged_data["vib_angle_diff_elbow_1"] 

    # Test hypothesis that data1 has higher or lower mean than data2 with a non-parametric paired test
    test_result_less = stats.wilcoxon(merged_data["vib_diff"], alternative="less")
    test_result_greater = stats.wilcoxon(merged_data["vib_diff"], alternative="greater")

    return test_result_less.pvalue, test_result_greater.pvalue

test_results = pd.DataFrame(columns=["muscle", "freq1", "freq2", "p_value_less", "p_value_greater"])

for muscle in [["BIClong"], ["TRIlat"]]:
    for freq1 in [20, 40, 60, 80, 100, 110, 130, 150, 170, 190]:
        for freq2 in [20, 40, 60, 80, 100, 110, 130, 150, 170, 190]:
            if freq1 >= freq2:
                continue
            p_less, p_greater = paired_test_vibration_freqs(df_all, muscle, freq1, freq2)
            test_results = pd.concat(
                [test_results, 
                pd.DataFrame([[muscle[0], freq1, freq2, p_less, p_greater]], columns=test_results.columns)]
            )

test_results.to_csv("figure_3_test_vib_freqs.csv", index=False)